In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install wandb
from kaggle_secrets import UserSecretsClient
import wandb
user_secrets = UserSecretsClient()

my_secret = user_secrets.get_secret("wandb_api_key") 

wandb.login(key=my_secret)

In [ ]:
CFG = {
    'model_name' : 'RESNET-34',
    'BATCH_SIZE' : 32,
    'LEARNING_RATE' : 0.001,
    'MOMENTUM' : 0.9,
    'STEP_SIZE' : 7,
    'GAMMA' : 0.1,
    'EPOCHS' : 15,
    'TRAIN_DIR' : '/kaggle/input/prasansa-data/DevanagariHandwrittenCharacterDataset/Train',
    'TEST_DIR' : '/kaggle/input/prasansa-data/DevanagariHandwrittenCharacterDataset/Test'
}

# Initialise run
run = wandb.init(project = 'devnagari_character_recog',
                 config = CFG,
                 save_code = True,
                 name='RESNET-34_train'
)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import copy
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import pandas as pd
from torch.utils.data import DataLoader, random_split, Dataset

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
# Hyperparameters
BATCH_SIZE = CFG['BATCH_SIZE']
LEARNING_RATE = CFG['LEARNING_RATE']
MOMENTUM = CFG['MOMENTUM']
STEP_SIZE = CFG['STEP_SIZE']
GAMMA = CFG['GAMMA']
EPOCHS = CFG['EPOCHS']

TRAIN_DIR = CFG['TRAIN_DIR']
TEST_DIR = CFG['TEST_DIR']

In [ ]:
# Custom Dataset wrapper for applying transforms
class TransformedSubset(Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform

    def __getitem__(self, index):
        x, y = self.subset[index]
        if self.transform:
            x = self.transform(x)
        return x, y

    def __len__(self):
        return len(self.subset)

In [ ]:
# Data transforms
augment_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset and split
full_dataset = datasets.ImageFolder(root=TRAIN_DIR)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_subset, val_subset = random_split(full_dataset, [train_size, val_size])

# Create datasets
no_augment_train = TransformedSubset(train_subset, test_transform)
augment_train = TransformedSubset(train_subset, augment_transform)
val_dataset = TransformedSubset(val_subset, test_transform)

# Create dataloaders
no_augment_loader = DataLoader(no_augment_train, BATCH_SIZE, shuffle=True)
augment_loader = DataLoader(augment_train, BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, BATCH_SIZE, shuffle=False)

num_classes = len(full_dataset.classes)


In [ ]:
print(len(full_dataset.classes))

In [ ]:
def create_model():
    model = models.resnet34(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
    return model

def train_model(model,train_loader, val_loader, name):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    # Tensor.cpu()
    criterion = nn.CrossEntropyLoss()    #TENSOR
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)
    scheduler = lr_scheduler.StepLR(optimizer, STEP_SIZE, GAMMA)

    best_acc = 0.0
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

    for epoch in range(EPOCHS):
        print(f'Epoch {epoch+1}/{EPOCHS}')
        model.train()
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            if isinstance(outputs, torch.nn.modules.container.ModuleDict):
                outputs = outputs.logits  # Extract only the main output
            loss = criterion(outputs, labels)

            # outputs = model(inputs)       #INCEPTIONOUTPUT
            # loss = criterion(outputs, labels) #EXPECTS TENSOR
            loss.backward()
            optimizer.step()
            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        scheduler.step()
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)
        history['train_loss'].append(epoch_loss)
        history['train_acc'].append(epoch_acc)

        print('hello')

        # Validation
        model.eval()
        val_loss = 0.0
        val_corrects = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)
                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)

        val_epoch_loss = val_loss / len(val_loader.dataset)
        val_epoch_acc = val_corrects.double() / len(val_loader.dataset)
        history['val_loss'].append(val_epoch_loss)
        history['val_acc'].append(val_epoch_acc)

        print(f'{name} Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        print(f'{name} Val Loss: {val_epoch_loss:.4f} Acc: {val_epoch_acc:.4f}\n')
        
        wandb.log({f'{name} Train Loss': epoch_loss})
        wandb.log({f'{name} Train Accuracy': epoch_acc.cpu().numpy()})
        wandb.log({f'{name} Val Loss': val_epoch_loss})
        wandb.log({f'{name} Val Accuracy': val_epoch_acc.cpu().numpy()})
        SAVE_PATH = '/kaggle/working/best_model_inceptionv3.torch'
        if val_epoch_acc > best_acc:
            
            best_acc = val_epoch_acc
            best_model = copy.deepcopy(model.state_dict())
            torch.save(best_model, SAVE_PATH)
            wandb.save(SAVE_PATH)

    model.load_state_dict(best_model)
    wandb.finish()
    return model, history


In [ ]:
# Train without augmentation
print("Training without augmentation...")
model_no_aug, hist_no_aug = train_model(model, no_augment_loader, val_loader, "No Aug")

In [ ]:
# Train with augmentation
print("\nTraining with augmentation...")
model_aug, hist_aug = train_model(augment_loader, val_loader, "Aug")

In [ ]:
# Plot results
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(hist_no_aug['train_loss'], label='No Aug Train')
plt.plot(hist_aug['train_loss'], label='Aug Train')
plt.plot(hist_no_aug['val_loss'], '--', label='No Aug Val')
plt.plot(hist_aug['val_loss'], '--', label='Aug Val')
plt.title('Loss Comparison')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(hist_no_aug['train_acc'], label='No Aug Train')
plt.plot(hist_aug['train_acc'], label='Aug Train')
plt.plot(hist_no_aug['val_acc'], '--', label='No Aug Val')
plt.plot(hist_aug['val_acc'], '--', label='Aug Val')
plt.title('Accuracy Comparison')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
plt.subplot(1, 2, 2)
plt.plot(hist_no_aug['train_acc'], label='No Aug Train')
plt.plot(hist_aug['train_acc'], label='Aug Train')
plt.plot(hist_no_aug['val_acc'], '--', label='No Aug Val')
plt.plot(hist_aug['val_acc'], '--', label='Aug Val')
plt.title('Accuracy Comparison')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Test evaluation
test_dataset = datasets.ImageFolder(TEST_DIR, test_transform)
test_loader = DataLoader(test_dataset, BATCH_SIZE, shuffle=False)


def evaluate(model, loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    all_preds = []
    all_labels = []
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    
    accuracy = 100 * correct / total
    return all_labels, all_preds, accuracy

# Evaluate best model
labels, preds, test_acc = evaluate(model_aug, test_loader)
print("Test Results:")
print(classification_report(labels, preds, target_names=test_dataset.classes))
print(f"Test Accuracy: {test_acc:.2f}%")


In [ ]:
# Compute the confusion matrix
cm = confusion_matrix(labels, preds)

# Plot the confusion matrix
plt.figure(figsize=(20, 20))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=test_dataset.classes, 
            yticklabels=test_dataset.classes)

# Labels and title
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Show the plot
plt.show()